Linear Regression 線性回歸

In [8]:
import sys
import pandas as pd
import numpy as np

data = pd.read_csv('./train.csv', header=None, encoding='big5')

In [9]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
1,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
2,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
3,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
4,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08


In [10]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,4321,4321,4321,4321,4321,4321,4321,4321,4321,4321,...,4321,4321,4321,4321,4321,4321,4321,4321,4321,4321
unique,241,2,19,369,361,351,355,353,342,356,...,423,411,409,423,405,374,366,374,382,370
top,2014/6/20,豐原,PM10,NR,NR,NR,NR,NR,NR,NR,...,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR
freq,18,4320,240,221,225,229,226,229,230,226,...,220,219,221,221,222,223,225,224,226,224


In [11]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

data.head()

,3,4,5,6,7,8,9,10,11,12,...,17,18,19,20,21,22,23,24,25,26
0,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
1,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
2,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
3,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
4,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08


In [14]:
raw_data

array([['0', '1', '2', ..., '21', '22', '23'],
       ['14', '14', '14', ..., '15', '15', '15'],
       ['1.8', '1.8', '1.8', ..., '1.8', '1.8', '1.8'],
       ...,
       ['36', '55', '72', ..., '118', '100', '105'],
       ['1.9', '2.4', '1.9', ..., '1.5', '2', '2'],
       ['0.7', '0.8', '1.8', ..., '1.6', '1.8', '2']], dtype=object)

In [13]:
day_data = {}
for day in range(20):
    sample = np.empty([18, 24])
    sample[:, day * 24: (day + 1) * 24] = raw_data[18 *(20 * day): 18 * (20 * day + 1), :]
    month_data[day] = sample